# Iterative based solvers

## Kronecker Product - Conjugate Gradient
### General algorithm
Rewrite $AX+XB=C$ as 
- $\mathcal{A}x = c$

where $\mathcal{A} = I \otimes A + B^* \otimes I$, $x = \text{vec}(X)$ and $c = \text{vec}(C)$

### TU + UT = F
Rewrite as
- $\mathcal{T}u = f$

where $\mathcal{T} = I \otimes T + T \otimes I$, $u = \text{vec}(U)$ and $f = \text{vec}(F)$

In [ ]:
import numpy as np
from scipy.sparse import kron
from scipy.sparse.linalg import cg
import time

def kron_prod_it(T, F, tol):
    
    num_iters = 0
    
    def callback(xk): #Used to count number of iterations
        nonlocal num_iters
        num_iters += 1
    
    start_time = time.time()
    F = np.reshape(F,-1)
    A = kron(np.eye(n),T) + kron(T.transpose(),np.eye(n)) #A = I kron T + T^t kron I
    kron_time = time.time() - start_time
    U, info = cg(A, F, tol=tol, callback=callback) #Solve using scipy sparse conjugate grad
    end_time = time.time()
    solve_time = end_time - kron_time - start_time
    total_time = end_time - start_time
    timings = [kron_time, solve_time, total_time]
    U = np.reshape(U,(n,n)) #Reshape U for plotting/computing error
    print(np.linalg.cond(A.todense()))
    return U, timings, num_iters

## Gradient Based Method
### General algorithm
Rewrite $AX + XB = C$ as:
- $AX = C-XB$
- $XB = C-AX$

Then define two recursive sequences:
- $X_k^{(1)} = X_{k-1}^{(1)} + \kappa A^T (C-AX_{k-1}^{(1)} - X_{k-1}^{(1)}B)$
- $X_k^{(2)} = X_{k-1}^{(2)} + \kappa (C-AX_{k-1}^{(2)} - X_{k-1}^{(2)}B)B^T$

where $\kappa$ is the relative step size. The $k^{\text{th}}$ approximate solution is then defined as the average of the above two iterative procedures
- $X_k = \frac{X_k^{(1)} + X_k^{(2)}}{2}$

To compute this an appropriate initial approximate solution $X_0$ is needed. The method converges if 
- $0 < \kappa < \frac{2}{\lambda_{\text{max}}(AA^T) + \lambda_{\text{max}}(B^TB)} $

where $\lambda_{\text{max}}(AA^T)$ is the max eigenvalue of $AA^T$

### TU+UT=F
Recursive sequences: (noting that $T^T = T$)
- $U_k^{(1)} = U_{k-1}^{(1)} + \kappa T(F-TU_{k-1}^{(1)} - U_{k-1}^{(1)}T)$
- $U_k^{(2)} = U_{k-1}^{(2)} + \kappa (F-TU_{k-1}^{(2)} - U_{k-1}^{(2)}T)T$

$k^{\text{th}}$ approximate solution $U_k$ is:
- $U_k = \frac{U_k^{(1)} + U_k^{(2)}}{2}$

Solution converges if:
- $0 < \kappa < \frac{1}{\lambda_{\text{max}}(T^2)} $


In [ ]:
import numpy as np
import time

def grad_based(T, F, tol):
    n = len(T)
    U = np.zeros([n,n])
   
    tol_mat = np.empty([n,n])
    for i in range(0,n):
        for j in range(0,n):
            tol_mat[i][j]=tol

    #Set U_1 & U_2 equal to initial guess
    U_1 = U
    U_2 = U
    
    start_time = time.time()

    #Calculate eigenvalues to check method converges
    eigvals = np.linalg.eig(np.matmul(T,T))[0]

    #Get maximum eigenvalue
    eig_max = np.max(eigvals)

    k = 1/(2*eig_max) #k must be smaller than 1/eig_max

    num_iters = 0
    while (np.absolute(np.matmul(T,U) + np.matmul(U,T) - F) > tol).any() == True:
        U_1 = U_1 + k*np.matmul(T,(F - np.matmul(T, U_1) - np.matmul(U_1,T)))
        U_2 = U_2 + k*np.matmul((F - np.matmul(T,U_2) - np.matmul(U_2,T)), T)

        U = (U_1+U_2)/2
        num_iters += 1 
        
    total_time = time.time() - start_time
    return U, total_time, num_iters





## Modified Conjugate Gradient
### General algorithm
1. Choose initial guess $X^{(0)}$ and calculate:
  - $R^{(0)} = C - AX^{(0)} - X^{(0)}B$
  - $Q^{(0)} = A^T R^{(0)} + R^{(0)} + R^{(0)} B^T$
  - $Z^{(0)} = Q^{(0)}$

2. If $R^{(k)} < \text{tol}$ or $Z^{(k)} < \text{tol}$, stop. Else, go to 3.

3. Calculate:
  - $\gamma_k = \frac{[R^{(k)}, R^{(k)}]}{[Z^{(k)}, Z^{(k)}]}$, where $[R,R] = \text{tr}(R^T R)$ is the trace of the matrix $R^T R$
  - $X^{(k+1)} = X^{(k)} + \gamma_k Z^{(k)}$
  - $R^{(k+1)} = C - AX^{(k+1)} - X^{(k+1)} B$
  - $Q^{(k+1)} = A^T R^{(k+1)} + R^{(k+1)} + R^{(k+1)} B^T$

and return to step 2.
  
###  TU + UT = F
1. Choose initial guess $U^{(0)}$ and calculate:
    - $R^{(0)} = F - TU^{(0)} - U^{(0)}T$
	- $Q^{(0)} = T R^{(0)} + R^{(0)} + R^{(0)} T$
	- $Z^{(0)} = Q^{(0)}$
2. If $R^{(k)} < \text{tol}$ or $Z^{(k)} < \text{tol}$, stop. Else, go to 3.
3. Calculate:
	- $\gamma_k = \frac{[R^{(k)}, R^{(k)}]}{[Z^{(k)}, Z^{(k)}]}$
	- $U^{(k+1)} = U^{(k)} + \gamma_k Z^{(k)}$
	- $R^{(k+1)} = F - TU^{(k+1)} - U^{(k+1)} T$
	- $Q^{(k+1)} = T R^{(k+1)} + R^{(k+1)} + R^{(k+1)} T$

and return to Step 2.

In [ ]:
import numpy as np
import time

def mcg(T, F, tol):
    n = len(T)
    U = np.zeros([n,n])
    
    start_time = time.time()
    R = F - np.matmul(T,U) - np.matmul(U,T)
    Q = np.matmul(T.transpose(),R) + R + np.matmul(R,T.transpose())
    Z = Q

    num_iters = 0
    if ((np.absolute(Z) > tol).any()) == True:
        while ((np.absolute(R) > tol).any() == True):
            gamma = np.trace(np.matmul(R.transpose(),R))/np.trace(np.matmul(Z.transpose(),Z))
            U = U + gamma*Z
            R = F - np.matmul(T,U) - np.matmul(U,T)
            Q = np.matmul(T.transpose(),R) + R + np.matmul(R,T.transpose())
            num_iters += 1
    total_time = time.time() - start_time
    return U, total_time, num_iters

## Preconditioned MCG
### General algorithm
1. Choose initial guess $X^{(0)}$ and calculate:
	- $R^{(0)} = -\widetilde{C} + X^{(0)} - \widetilde{A}X^{(0)} \widetilde{B}$
	- $Q^{(0)} = \widetilde{A}^T R^{(0)}\widetilde{B}^T + R^{(0)}$
	- $Z^{(0)} = Q^{(0)}$
2. If $R^{(k)} < \text{tol}$ or $Z^{(k)} < \text{tol}$, stop. Else, go to 3.
3. Calculate:
	- $ \gamma_k = \frac{[R^{(k)}, R^{(k)}]}{[Z^{(k)}, Z^{(k)}]}$
	- $X^{(k+1)} = X^{(k)} + \gamma_k Z^{(k)} $
	- $R^{(k+1)} = -\widetilde{C} + X^{(k+1)} - \widetilde{A}X^{(k+1)} \widetilde{B}$
	- $Q^{(k+1)} = \widetilde{A}^T R^{(k+1)}\widetilde{B}^T - R^{(k+1)}$
	- $\eta_k = \frac{[R^{(k+1)}, R^{(k+1)}]}{[R^{(k)}, R^{(k)}]}$ 
	- $Z^{(k+1)} = Q^{(k+1)} + \eta_k Z^{(k)}$

and return to Step 2.

### TU + UT = F

1. Choose initial guess $U^{(0)}$ and calculate:
	- $R^{(0)} = -\widetilde{F} + U^{(0)} - \widetilde{T}U^{(0)} \widetilde{T}$
	- $Q^{(0)} = \widetilde{T} R^{(0)}\widetilde{T} + R^{(0)}$
	- $Z^{(0)} = Q^{(0)}$
2. If $R^{(k)} < \text{tol}$ or $Z^{(k)} < \text{tol}$, stop. Else, go to 3.
3. Calculate:
	- $ \gamma_k = \frac{[R^{(k)}, R^{(k)}]}{[Z^{(k)}, Z^{(k)}]}$
	- $U^{(k+1)} = U^{(k)} + \gamma_k Z^{(k)} $
	- $R^{(k+1)} = -\widetilde{F} + U^{(k+1)} - \widetilde{T}U^{(k+1)} \widetilde{T}$
	- $Q^{(k+1)} = \widetilde{T} R^{(k+1)}\widetilde{T} - R^{(k+1)}$
	- $\eta_k = \frac{[R^{(k+1)}, R^{(k+1)}]}{[R^{(k)}, R^{(k)}]}$ 
	- $Z^{(k+1)} = Q^{(k+1)} + \eta_k Z^{(k)}$

and return to Step 2.

In [ ]:
import numpy as np
import time

def mcg_pre(T, F, tol):
    n = len(T)
    U = np.zeros([n,n])
    
    start_time = time.time()
    norm = np.sqrt(np.trace(np.matmul(T.transpose(),T)))
    alpha = -np.sqrt(n)/norm
    Y = np.linalg.inv((alpha*T - np.eye(n)))
    T_hat = np.eye(n) + 2*Y
    F_hat = 2*alpha*(np.matmul(np.matmul(Y,-F),Y))
    R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
    Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
    Z = Q    

    num_iters = 0
#     if (np.any(np.absolute(Z) > tol).any()) == True:
    while ((np.absolute(R) > tol).any() == True):
        gamma = np.trace(np.matmul(R.transpose(),R))/np.trace(np.matmul(Z.transpose(),Z))
        eta_0 = np.trace(np.matmul(R.transpose(),R))
        U = U + gamma*Z
        R = -F_hat + U - np.matmul(np.matmul(T_hat,U),T_hat)
        Q = np.matmul(np.matmul(T_hat.transpose(),R),T_hat.transpose()) - R
        eta = np.trace(np.matmul(R.transpose(),R))/eta_0
        Z = Q + eta*Z
        num_iters += 1
    total_time = time.time() - start_time
    return U, total_time, num_iters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def plot_sol(X,Y,U):
    plt.figure(0)
    xline = np.reshape(X, -1)
    yline = np.reshape(Y, -1)
    zline = np.reshape(U, -1)
    plt.imshow(U, extent=[0,1,0,1], origin='lower')
    plt.colorbar()
    plt.axis(aspect='image')
    plt.xlabel('x')
    plt.ylabel('y')

def compute_err(U, U_exact):
    n = len(U)
    err_inf = 0
    err_sq = 0
    for i in range(0,n):
        for j in range(0,n):
            err_sq += np.absolute(U_exact[i][j] - U[i][j])**2
            if np.absolute(U_exact[i][j] - U[i][j]) > err_inf:
                err_inf = np.absolute(U_exact[i][j] - U[i][j])
        
    err_sq = (err_sq * h**2)**0.5
    return err_inf, err_sq

In [ ]:
from scipy.sparse import diags

#Define parameters
n = 125 #number of internal nodes in each direction (number of unknowns)
h = 1/(n+1) #step size

U = np.zeros([n,n])

#Define x and y as arrays between 0 and 1 with n evenly spaced points (internal nodes)
x = np.linspace(h, 1-h, n)
y = np.linspace(h, 1-h, n)

#Create internal mesh (excludes boundaries)
X, Y = np.meshgrid(x, y, indexing='ij')

#Define F 
F = 2 * np.pi**2 * np.sin(np.pi*X) * np.sin(np.pi*Y)  

#Define tridiagonal matrix T
diagonals = [[-2],[1],[1]]
T = np.multiply((-1)/(h**2), diags(diagonals, [0, -1, 1], shape=(n, n)).toarray())

#Compute exact solution for comparison
U_exact = np.sin(np.pi*X) * np.sin(np.pi*Y)

#Solve system
U, total_time, num_iters = kron_prod_it(T,F,1e-9)

err_inf, err_sq = compute_err(U, U_exact)
print('Time taken:', total_time, '\nNumber of iterations:', num_iters)
print('Error inf:', err_inf, '\nError sq:', err_sq)

